# Adding redundant constraints:

In [ ]:
import numpy as np
import random 
from random import randint
import math  
from cplex.callbacks import UserCutCallback
import cplex  as CPX
import cplex.callbacks as CPX_CB
import sys
import time
from numpy.linalg import matrix_rank
import pandas as pd

# Defining the parameters:

In [ ]:
# example location
examples=['data/NewExample1']
# percentage of adding reduntant constraints:
RedundantPer=[0.1,0.25,0.5]

# Adding redundant contraints:

In [ ]:
for example in examples:
    for k in RedundantPer:          
        NonNegativeRed=1
        c = CPX.Cplex(example+'.lp')
        m=c.linear_constraints.get_num()
        n=c.variables.get_num()
        my_coef=[]
        for i in range(m):
            my_row=[]
            for j in range(n):
                my_row.append(c.linear_constraints.get_coefficients(i,j))
            my_coef.append(my_row)

        for i in range (n):
            tmp=[]
            for j in range (n):
                if i==j:
                    tmp.append(-1)
                else:
                    tmp.append(0)
            my_coef.append(tmp) 
        FinalCoeff=my_coef[:] 

        NumNewConst=math.floor(k*len (my_coef))
        for j in range (int(NumNewConst)):  
            new_cons=np.zeros((1,n))
            for i in range (len(my_coef)):
                NonNeg=random.uniform(0, NonNegativeRed) 
                newrow=np.array(my_coef[i])*NonNeg
                new_cons=np.add(new_cons,newrow)   
            new_cons=list((new_cons)[0])
            sumline=[]
            sumline.append([range(n),new_cons])    
            c.linear_constraints.add(lin_expr = sumline, senses =  "L",
                                    rhs = [0], names=(["c"+str(m+j)]))
        c.write(example+'Redundant'+str(k)+'.lp')    